In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ s', r = env(s, a)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=states.dtype)
    next_states_logits, Qlogits = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                      action_size=action_size, state_size=state_size)
    ################################################ a = act(s)
    ################################################ ~s', Q = ~env(s, a)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    next_states_logits_, Qlogits_ = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                        action_size=action_size, state_size=state_size, reuse=True)
    ###########################################################  GQN+GAN
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, # GQN/GAN
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits_, # GQN/GAN
                                                                     labels=next_states_labels))
    aloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits_, # GQN/GAN
                                                                   labels=next_states_labels))
    ###########################################################  GAN-QProb
    Qlogits = tf.reshape(Qlogits, shape=[-1])
    Qlogits_ = tf.reshape(Qlogits_, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits, # GAN/DQN
                                                                    labels=rates))
#     eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits_, # GAN/DQN
#                                                                      labels=rates))
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits_, # GAN/DQN
                                                                    labels=tf.zeros_like(rates)))
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits_, # GAN/DQN
                                                                    labels=tf.ones_like(rates)))
    ################################################################ Q = r + (gamma * Q'* dones)
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _next_states_logits_, nextQlogits_ = Env(actions=next_actions_logits, states=next_states, 
                                             hidden_size=hidden_size, action_size=action_size, 
                                             state_size=state_size, reuse=True)
    ##################################################################### DQN+GAN-Qvalue
    targetQs = rewards + (gamma * tf.reshape(nextQlogits_, shape=[-1]) * dones)
#     targetQs_labels = tf.nn.sigmoid(targetQs)
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits, # DQN
#                                                                     labels=targetQs_labels)) # min MCE
#     eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits_, # DQN
#                                                                      labels=targetQs_labels)) # min MCE
    eloss += tf.reduce_mean(tf.square(Qlogits - targetQs))
    eloss += tf.reduce_mean(tf.square(Qlogits_ - rewards))
    aloss += -tf.reduce_mean(Qlogits_)
    return actions_logits, aloss, eloss

In [7]:
def model_opt(a_loss, e_loss, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
    return a_opt, e_opt

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt = model_opt(a_loss=self.a_loss, e_loss=self.e_loss,
                                           a_learning_rate=a_learning_rate,
                                           e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
        return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [14]:
def minibatch_online(memory):
    batch = memory.buffer
    #for idx in range(memory_size// batch_size):
    while True:
        idx = np.random.choice(np.arange(memory_size// batch_size))
        states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
        states = states[rates >= np.max(rates)]
        actions = actions[rates >= np.max(rates)]
        next_states = next_states[rates >= np.max(rates)]
        rewards = rewards[rates >= np.max(rates)]
        dones = dones[rates >= np.max(rates)]
        rates = rates[rates >= np.max(rates)]
        if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
            break
    return states, actions, next_states, rewards, dones, rates

In [15]:
# def minibatch(memory):
#     while True:
#         batch = memory.sample(batch_size=batch_size)
#         rates = np.array([each[5] for each in batch])
#         states = np.array([each[0] for each in batch])[rates > 0]
#         actions = np.array([each[1] for each in batch])[rates > 0]
#         next_states = np.array([each[2] for each in batch])[rates > 0]
#         rewards = np.array([each[3] for each in batch])[rates > 0]
#         dones = np.array([each[4] for each in batch])[rates > 0]
#         rates = np.array([each[5] for each in batch])[rates > 0]
#         if len(dones) == batch_size:
#             break
#     return states, actions, next_states, rewards, dones, rates

In [16]:
# def minibatch_online(memory):
#     batch = memory.buffer
#     rates = np.array([each[5] for each in batch])
#     states = np.array([each[0] for each in batch])[rates == -1]
#     actions = np.array([each[1] for each in batch])[rates == -1]
#     next_states = np.array([each[2] for each in batch])[rates == -1]
#     rewards = np.array([each[3] for each in batch])[rates == -1]
#     dones = np.array([each[4] for each in batch])[rates == -1]
#     rates = np.array([each[5] for each in batch])[rates == -1]
#     return states, actions, next_states, rewards, dones, rates

In [17]:
# states, actions, next_states, rewards, dones, rates_on = minibatch_online(memory=memory)
states, actions, next_states, rewards, dones, rates = minibatch_online(memory=memory)

In [18]:
rates

array([0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092,
       0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092,
       0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092,
       0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092,
       0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092, 0.092,
       0.092])

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch = [], []
        total_reward = 0
        state = env.reset()
        rate = -1
        
        # Episode steps
        for num_step in range(1111111111111):
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            total_reward += reward
            state = next_state
            
            # Online Training/on-policy
            states, actions, next_states, rewards, dones, rates = minibatch_online(memory=memory)
            feed_dict = {model.states: states, model.actions: actions, model.next_states: next_states,
                         model.rewards: rewards, model.dones: dones, model.rates: rates}
            eloss, _ = sess.run([model.e_loss, model.e_opt], feed_dict)
            aloss, _ = sess.run([model.a_loss, model.a_opt], feed_dict)
            eloss_batch.append(eloss)
            aloss_batch.append(aloss)
            if done is True:
                # Rating the latest played episode
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                break
                

        # Offline Training/off-policy
        # EMPTYYYYYYYYYYYYY#####
        # EMPTYYYYYYYYYYYYY#####
        # EMPTYYYYYYYYYYYYY#####
        # EMPTYYYYYYYYYYYYY#####
        
        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:9.0000 R:9.0000 rate:0.0180 aloss:1.5266 eloss:3.6561
Episode:1 meanR:9.5000 R:10.0000 rate:0.0200 aloss:1.5172 eloss:3.6303
Episode:2 meanR:9.0000 R:8.0000 rate:0.0160 aloss:1.5289 eloss:3.6736
Episode:3 meanR:9.2500 R:10.0000 rate:0.0200 aloss:1.4355 eloss:3.4441
Episode:4 meanR:9.4000 R:10.0000 rate:0.0200 aloss:1.4822 eloss:3.5653
Episode:5 meanR:9.5000 R:10.0000 rate:0.0200 aloss:1.5353 eloss:3.6732
Episode:6 meanR:9.4286 R:9.0000 rate:0.0180 aloss:1.5839 eloss:3.8101
Episode:7 meanR:9.3750 R:9.0000 rate:0.0180 aloss:1.4504 eloss:3.4884
Episode:8 meanR:9.4444 R:10.0000 rate:0.0200 aloss:1.4224 eloss:3.4307
Episode:9 meanR:9.5000 R:10.0000 rate:0.0200 aloss:1.4853 eloss:3.5454
Episode:10 meanR:9.3636 R:8.0000 rate:0.0160 aloss:1.3644 eloss:3.3159
Episode:11 meanR:9.5000 R:11.0000 rate:0.0220 aloss:1.5314 eloss:3.6817
Episode:12 meanR:9.4615 R:9.0000 rate:0.0180 aloss:1.3885 eloss:3.3731
Episode:13 meanR:9.4286 R:9.0000 rate:0.0180 aloss:1.3875 eloss:3.3616
Episode:1

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
